In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch
from radarnet import RadarNet
import os
from matplotlib.animation import FuncAnimation
from IPython.display import HTML


In [2]:
# retrieve 10 radar images at one time
tsize=10

def retrieve_imgs(path):
    global tsize
    all_imgs= os.listdir(path)
    n= np.random.randint(0, len(all_imgs))
    while True:
        imgs_input= []
        imgs_target= []
        date_time= []
        for it in range(tsize):
            img_in= np.array(Image.open(os.path.join(radar_img_path, all_imgs[n+it])))
            img_out= np.array(Image.open(os.path.join(radar_img_path, all_imgs[n+it+1])))
            # store time
            date_str= all_imgs[n+it+1].split('.')[0]
            # normalization
            img_in= img_in/img_in.max()
            img_out= img_out/img_out.max()
            imgs_input.append(img_in)
            imgs_target.append(img_out)
            date_time.append(date_str)
        
        return date_time, np.array(imgs_input)[np.newaxis,:,np.newaxis,400:600,400:600],\
                np.array(imgs_target)[np.newaxis,:,np.newaxis,400:600,400:600]
            

In [3]:
radar_img_path= '../ModelForecast/RadarRaw'
time, test_img_in, test_img_out= retrieve_imgs(radar_img_path)
model_path= 'logging/newest.pth'
print(test_img_in.shape)
# normalization
test_img_in= torch.Tensor(test_img_in)

(1, 10, 1, 200, 200)


In [4]:
model= RadarNet()
model.load_state_dict(torch.load(model_path, map_location='cpu'))
model.eval()
out= model(test_img_in)
print(out.shape)

torch.Size([1, 10, 1, 200, 200])


In [5]:
test_img_out.shape

(1, 10, 1, 200, 200)

In [10]:
out_arr= out.detach().numpy().squeeze()
test_img_out= test_img_out.squeeze()

fig, ax= plt.subplots(1,2,figsize=(10,8))

def frame():
    global tsize, out_arr, test_img_out, time
    for it in range(tsize):
        yield time[it], out_arr[it], test_img_out[it]
    
def update(args):
    time= args[0]
    pred= args[1]
    target= args[2]
    im1=ax[0].imshow(pred)
    ax[0].set_title(time+' prediction')
    im2=ax[1].imshow(target)
    ax[1].set_title(time+' observation')
    plt.close()
    

ani= FuncAnimation(fig,update,frame)


In [11]:
HTML(ani.to_html5_video())

In [12]:
# save results
ani.save('201811230600-predict.gif',writer='imagemagick')

MovieWriter imagemagick unavailable. Trying to use pillow instead.
